# Studi Kasus Beer Production

# PROSES PERAMALAN MODEL SAMPAI TAHUN 2023 JANUARI

## PROFILE KELOMPOK


NPM KETUA : 200411100153

NAMA KETUA : FIKRI AINUN NAJIB

NPM ANGGOTA : 200411100166

NAMA ANGGOTA : GHANIY ALBAR RASYIDI KUSUMA

GITHUB : https://github.com/LALA09-erha/kolaborasipro

DATASET : https://www.kaggle.com/datasets/shenba/time-series-datasets?select=monthly-beer-production-in-austr.csv


KELAS PSD C

Inisialisasi Libary dan beberapa Model

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
import datetime
from dateutil.relativedelta import relativedelta




Pengambilan data

In [33]:
data = pd.read_csv("../Data/monthly-beer-production-in-austr.csv")

# add new data input to the dataframe
data = data ['Monthly beer production']


Split Data

In [34]:
data_training,data_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False, random_state=42)

In [35]:
data_training_new = pd.DataFrame(data_training)
data_test_new = pd.DataFrame(data_test)

fungsi Cross Validation

In [36]:
# Cross Validation
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

Eksekusi Export Data Ke CSV

In [37]:
print(pd.date_range(start = '1995-9-1',end='2023-1-1', freq='M'))


DatetimeIndex(['1995-09-30', '1995-10-31', '1995-11-30', '1995-12-31',
               '1996-01-31', '1996-02-29', '1996-03-31', '1996-04-30',
               '1996-05-31', '1996-06-30',
               ...
               '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30',
               '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31',
               '2022-11-30', '2022-12-31'],
              dtype='datetime64[ns]', length=328, freq='M')


In [38]:
# PROSES MEMASUKKAN DATA PREDIKSI KE CSV
# membuat dataframe kosong dengan 3 kolom
data_final = pd.DataFrame(columns=['Month','KNN' , 'SVR' , 'RF'])
# berawal pada 1995-september
start_date = datetime.date(1995, 9, 1)
 
# berakhir pada 2023-januari 
end_date = datetime.date(2023, 12, 1)

# loop setiap bulan
delta = relativedelta(months=1)
while start_date <= end_date:
  tanggal = start_date.strftime("%Y-%m")
  # Preprosssing data
  sc = MinMaxScaler()
  train_scaled = sc.fit_transform(data_training_new)
  test_scaled = sc.transform(data_test_new)

  # Proses Pemanggilan Cross Validation 
  X_train, y_train = split_sequence(train_scaled, 6) 
  X_test, y_test = split_sequence(test_scaled, 6) 
  # membuat kolom otomatis
  colom = []
  for c in range(6):
      if(c==0):
          colom.insert(0,"Xt")
      else:
          colom.insert(0,f"Xt-{c}")



  # # Masukkan ke dataframe hasil dari split  squens
  # UNTUK MEMISAHKAN X TRAIN, YTRAIN dan TEST
  newX_train = pd.DataFrame(np.reshape(X_train, (len(X_train) , 6)),columns=colom)
  newY_train = pd.DataFrame(y_train, columns=['Output'])
  new_data_train = pd.concat([newX_train,newY_train],axis=1)
  newX_test = pd.DataFrame(np.reshape(X_test, (len(X_test) , 6)),columns=colom)
  newY_test = pd.DataFrame(y_test, columns=['Output'])
  new_data_test = pd.concat([newX_test,newY_test],axis=1)

  # EKSEKUSI KE model
  # predict knn
  knn = KNeighborsRegressor(n_neighbors = 4)
  knn.fit(newX_train,newY_train)

  # Proses Prediksi
  y_pred = knn.predict(newX_test)
  y_pred_shape = y_pred.reshape(-1,1)
  y_pred_inverse_knn = sc.inverse_transform(y_pred_shape)
  

  # Predict SVR dengan Bagging Regressor
  regr = BaggingRegressor(SVR(), n_estimators=2, random_state=0).fit(newX_train, newY_train.values.ravel())

  # Proses Prediksi
  y_pred = regr.predict(newX_test)
  y_pred_shape = y_pred.reshape(-1,1)
  y_pred_inverse_svr = sc.inverse_transform(y_pred_shape)
  
  

  # Predict dengan from RandomForestRegressor
  regr = RandomForestRegressor(max_depth=8, random_state=0).fit(newX_train, newY_train.values.ravel())

  # Proses Prediksi
  y_pred = regr.predict(newX_test)
  y_pred_shape = y_pred.reshape(-1,1)
  y_pred_inverse_rf = sc.inverse_transform(y_pred_shape)
  
   ## Proses Memasukkan Y_pred terakhir ke dataframe
  data_final = pd.concat([data_final,pd.DataFrame({'Month':tanggal,'KNN':np.round(y_pred_inverse_knn[-1]),'SVR':np.round(y_pred_inverse_svr[-1]),'RF':np.round(y_pred_inverse_rf[-1])})],axis=0)
  
  # mean dari 3 model dimasukkan ke dataframe data_test_new 
  data_test_new = pd.concat([data_test_new,pd.DataFrame({'Monthly beer production':np.round(np.mean([y_pred_inverse_knn[-1],y_pred_inverse_svr[-1],y_pred_inverse_rf[-1]]))},index=  [0] )],axis=0)
  start_date += delta

# save data to csv
data_final.to_csv('../Data/prediksijanuari2023.csv', index=False)
data_test_new.to_csv('../Data/datatestjanuari2023.csv', index=False)

